In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10,5))
import seaborn as sns
sns.set_context("paper")

import poissonlearning as pl

import storage
import plotting

plotting.setup(latex=True)

%load_ext autoreload
%autoreload 2

%matplotlib qt

In [2]:
SAVE_PLOTS = True
NUM_PLOTTING_POINTS = 1000

### Convenience functions

In [3]:
def compute_errors(experiments):
    for experiment in experiments:
        solution = experiment["solution"]

        reference_experiments = list(
            filter(
                lambda x: 
                    x["seed"] == experiment["seed"] 
                    and x["n"] == experiment["n"]
                    and x["bump"] == experiment["bump"]
                    and x["eps"] == experiment["eps"],
                experiments
            )
        )
        reference = max(reference_experiments, key=lambda x: x["p"])["solution"]

        error_L1 = np.abs(solution - reference).mean()
        experiment["error_L1"] = error_L1

In [4]:
experiments = storage.load_results(name="p_line", folder="../results")
compute_errors(experiments)

### Visualisation for various p

In [5]:
selected_experiments = list(
    filter(
        lambda x: 
            x["seed"] == 0 
            and x["n"] == 5000,
        experiments
    )
)
selected_experiments.sort(key=lambda x: x["p"])

In [6]:
sample_size = NUM_PLOTTING_POINTS

fig_results, ax_results = plt.subplots(1, 1, figsize=(10,4))
colors = plotting.get_plot_colors(n=len(selected_experiments))

for i, ex in enumerate(selected_experiments, start=0):
    sample = ex["solution"].sample(sample_size, random_state=1).sort_index()
    ax_results.plot(sample, label=f"p={ex['p']}; n={ex['n']}", c=colors[i])

ax_results.legend()
ax_results.grid(linestyle="dashed")
fig_results.tight_layout()

### Plot of error to largest p

In [79]:
n = 30000
selected_experiments = {
    n : list(
        filter(
            lambda x: 
                x["n"] == n
                and x["bump"] == "dirac",
            experiments
        )
    )
    for n in [10000, 20000, 30000]
}

p_error = list(set([e["p"] for e in list(selected_experiments.values())[0]]))
p_error = sorted(p_error)
p_max = max(p_error)
error = {bump: {} for bump in selected_experiments.keys()}
for bump, experiments_with_bump in selected_experiments.items():
    for p in p_error:
        if np.isclose(p, p_max):
            continue

        ex = list(filter(lambda x: np.isclose(x["p"], p), experiments_with_bump))
        error[bump][p] = {}
        error[bump][p]["mean"] = np.mean([e["error_L1"] for e in ex])
        error[bump][p]["max"] = np.max([e["error_L1"] for e in ex])
        error[bump][p]["min"] = np.min([e["error_L1"] for e in ex])

ValueError: max() arg is an empty sequence

In [ ]:
fig, ax = plt.subplots(1, 1)

colors = plotting.get_plot_colors(len(selected_experiments))
for i, (bump, e) in enumerate(error.items()):
    plotting.error_plot({bump: e}, ax, c=colors[i])

    ax.legend()
    ax.set_xlabel(fr"$p$")
    ax.set_ylabel(r"$\lVert u^p - u^{100} \rVert_1$")

if SAVE_PLOTS:
    fig.savefig(f"../plots/p_one_circle_error_largest_p.pdf", bbox_inches="tight")